In [1]:
import pandas as pd
import pickle
import numpy as np
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import sys
BASE_DIR = '.'
sys.path.append(os.path.join(BASE_DIR, '../../config'))
sys.path.append(os.path.join(BASE_DIR, '../model'))
sys.path.append(os.path.join(BASE_DIR, '../utils'))
from config import *
import datatable as dt

In [3]:
from sklearn.preprocessing import MinMaxScaler
user_emb1 = np.load(FEATURE_PATH + '/user_emb_normal_100.npy')


In [6]:
user_emb1[:, 0]


array([-1.11397691, -1.72281198,  0.17986505, ..., -0.40411964,
       -0.70699116, -0.23048743])

In [7]:
mms = MinMaxScaler(feature_range=(0, 1))
user = mms.fit_transform(user_emb1)

In [10]:
user[:, 0]


array([0.38518843, 0.24067623, 0.69229288, ..., 0.55367912, 0.48178998,
       0.59489221])

In [8]:
user_emb = pd.read_csv(FEATURE_PATH + '/user_emb50_weighted_action.csv')
tag = pd.read_csv(FEATURE_PATH + '/feed_info_tags_keys_des.csv')

In [42]:
tag_v = tag[[col for col in tag.columns if 'manual_tag_' in col]].values[:, 1:]
key_v = tag[[col for col in tag.columns if 'manual_key_' in col]].values
des_v = tag[[col for col in tag.columns if 'des_' in col]].values

In [50]:
tag_cnt = (tag_v != 0).sum(axis=1)
key_cnt = (key_v != 0).sum(axis=1)
des_cnt = (des_v != 0).sum(axis=1)

In [53]:
tag['tag_seq_len'] = tag_cnt
tag['key_seq_len'] = key_cnt
tag['des_seq_len'] = des_cnt

In [61]:
tag.to_csv(FEATURE_PATH + '/feed_info_tags_keys_des_seq_len.csv', index=False)

# user_emb

In [19]:
user_emb = pd.read_csv(FEATURE_PATH + '/user_emb100_weighted_action.csv')
user_emb

,userid,userid_of_feedid_emb_0,userid_of_feedid_emb_1,userid_of_feedid_emb_2,userid_of_feedid_emb_3,userid_of_feedid_emb_4,userid_of_feedid_emb_5,userid_of_feedid_emb_6,userid_of_feedid_emb_7,userid_of_feedid_emb_8,...,userid_of_feedid_emb_adjust_90,userid_of_feedid_emb_adjust_91,userid_of_feedid_emb_adjust_92,userid_of_feedid_emb_adjust_93,userid_of_feedid_emb_adjust_94,userid_of_feedid_emb_adjust_95,userid_of_feedid_emb_adjust_96,userid_of_feedid_emb_adjust_97,userid_of_feedid_emb_adjust_98,userid_of_feedid_emb_adjust_99
0,0,-1.113977,0.241492,0.048213,-0.084308,0.256532,0.387957,0.031632,0.763239,-0.311876,...,0.410476,0.679618,-0.155956,0.401830,-0.045820,0.394214,0.574962,-0.130230,-0.218960,0.356462
1,1,-1.722812,0.365271,-0.100460,-0.458344,-0.198187,0.239094,0.787418,0.977460,-0.217585,...,0.495657,0.851455,-0.129618,0.363159,-0.078068,0.334818,0.615420,-0.065902,-0.438668,0.302616
2,2,0.179865,0.515263,-0.488721,-0.155068,-0.220233,-0.740641,-1.048718,0.402200,-0.389355,...,0.672861,0.562221,-0.197561,-0.050075,0.395725,0.504234,0.233693,-0.009448,0.290675,-0.003992
3,3,-0.425055,-0.783106,0.082858,0.272062,0.252502,-0.183822,0.566883,-0.665986,-1.081161,...,0.058976,-0.070467,0.302878,-0.014678,0.381794,-0.102101,0.635307,-0.200736,0.372117,-0.708390
4,4,-0.689475,-0.778582,0.108032,0.029460,0.307563,-0.680754,0.806862,0.811297,-1.016329,...,0.023232,-0.251008,0.374487,0.057383,0.742515,0.072763,0.538206,-0.236182,0.407435,-0.670394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,250244,-0.368459,0.090223,-0.162723,0.647917,-0.255630,-0.951890,0.291639,0.779932,-0.456490,...,0.430337,0.736456,-0.169574,0.140287,-0.064411,0.261294,0.594854,-0.029613,0.025466,-0.097663
199995,250245,-0.587635,-0.646653,0.204161,0.073653,0.425683,-0.735139,0.770885,0.223822,-0.750307,...,0.113847,0.024089,0.501555,-0.265227,0.365113,-0.172466,0.566369,-0.289242,0.312102,-0.679109
199996,250246,-0.404120,-0.593984,-0.035975,-0.193814,0.434354,-0.781328,0.174258,0.360331,-0.171284,...,0.556639,0.432765,0.256078,-0.368968,0.448810,0.130307,0.237158,-0.127244,0.284097,-0.316457
199997,250247,-0.706991,-1.007519,0.024953,-0.007564,1.117876,-0.556772,1.570330,1.646933,-1.080616,...,0.024124,-0.194665,0.709288,-0.314613,0.667570,-0.174432,0.487626,-0.450120,0.420351,-0.811125


In [20]:
from sklearn.preprocessing import LabelEncoder

In [21]:
lbe = LabelEncoder()
lbe.fit(user_emb['userid'])
user_emb['userid'] = lbe.transform(user_emb['userid'])
user_emb.sort_values('userid', inplace=True)
user_emb.to_csv(FEATURE_PATH + '/mapped_user_emb_100_action_weighted.csv', index=False)

In [22]:
user_weight = user_emb.values[:,1:]
np.save(FEATURE_PATH + '/user_emb_100.npy', user_weight)

pkl = open(FEATURE_PATH + '/user_encoder.pkl', 'wb')
pickle.dump(lbe, pkl)
pkl.close()

In [23]:
emb1 = user_emb[['userid_of_feedid_emb_' + str(col) for col in range(100)]]
emb2 = user_emb[['userid_of_feedid_emb_adjust_' + str(col) for col in range(100)]]
np.save(FEATURE_PATH + '/user_emb_normal_100.npy', emb1)
np.save(FEATURE_PATH + '/user_emb_adjust_100.npy', emb2)

# feed emb

In [2]:
feed_emb = pd.read_csv(FEATURE_PATH + '/feed_embeddings_PCA100.csv')

In [3]:
feed_emb

,feedid,feed_embed_0,feed_embed_1,feed_embed_2,feed_embed_3,feed_embed_4,feed_embed_5,feed_embed_6,feed_embed_7,feed_embed_8,...,feed_embed_90,feed_embed_91,feed_embed_92,feed_embed_93,feed_embed_94,feed_embed_95,feed_embed_96,feed_embed_97,feed_embed_98,feed_embed_99
0,46022,0.032836,0.147723,0.027682,0.042132,-0.046634,0.032975,-0.034102,-0.048031,0.055487,...,0.003843,-0.031071,-0.008392,-0.021762,0.066757,0.071957,0.016586,-0.007608,0.069531,-0.040490
1,73903,-0.083487,0.173496,0.066599,-0.153011,-0.064012,0.015190,0.078975,0.061987,0.135770,...,-0.062963,-0.044430,0.044232,-0.090950,-0.030405,-0.062874,0.061384,-0.051264,-0.058863,0.071021
2,88646,0.033206,0.279637,-0.050914,-0.143481,-0.142618,-0.042491,-0.065638,0.052453,0.097598,...,0.041206,0.032397,0.041943,0.020291,-0.000868,0.091867,-0.001376,-0.056273,0.003454,-0.026046
3,24381,-0.144167,0.052527,-0.052714,-0.091405,0.163124,-0.038020,-0.072176,0.051027,0.137133,...,0.102879,-0.039342,-0.010831,-0.043872,0.087790,-0.042885,-0.071113,0.009889,-0.030981,0.019292
4,41542,-0.080061,0.040469,-0.005289,-0.152660,-0.024023,-0.062427,0.048908,0.035020,-0.038812,...,-0.144718,-0.023604,0.049973,-0.034466,0.020377,-0.075783,0.044173,0.061038,0.015109,-0.022412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106439,14792,-0.089543,-0.079452,0.067003,0.075939,0.094766,0.022104,0.032946,0.056212,0.052638,...,-0.058668,0.105036,-0.032666,0.033746,0.032897,-0.036763,0.003978,-0.002033,0.088895,0.019182
106440,42659,-0.069948,-0.114559,-0.053785,-0.061532,-0.005273,0.011619,0.047215,-0.124235,-0.102613,...,-0.013687,-0.149165,-0.003609,-0.037381,0.054318,-0.019028,-0.027890,-0.080203,0.002516,0.014117
106441,66410,-0.035552,-0.135890,0.021521,0.123782,0.021077,0.031301,-0.092861,0.005743,-0.088794,...,-0.103734,-0.178717,0.004056,0.021170,0.042306,-0.009751,0.055866,-0.052091,0.048184,0.003105
106442,7919,-0.037999,0.049988,-0.039765,0.076312,-0.063102,-0.168693,-0.001434,0.076829,0.013659,...,0.047147,0.043514,-0.084366,0.003480,-0.029920,0.014009,-0.051053,0.012264,-0.013668,-0.055354


In [4]:
all_emb = feed_emb.values[:, 1:]

In [98]:
gen = all_emb.mean(axis=0)

In [9]:
embs = np.zeros((feed_emb['feedid'].max() + 1, 100))

In [10]:
feed_emb.sort_values('feedid', inplace=True)

In [11]:
feed_emb

,feedid,feed_embed_0,feed_embed_1,feed_embed_2,feed_embed_3,feed_embed_4,feed_embed_5,feed_embed_6,feed_embed_7,feed_embed_8,...,feed_embed_90,feed_embed_91,feed_embed_92,feed_embed_93,feed_embed_94,feed_embed_95,feed_embed_96,feed_embed_97,feed_embed_98,feed_embed_99
99412,0,-0.111198,-0.126916,0.116095,0.147177,0.053560,-0.078009,-0.155295,0.076652,-0.161195,...,-0.044100,0.082607,0.036617,0.018875,-0.028116,0.017381,0.007060,-0.008716,-0.001330,-0.039618
13109,1,0.005251,0.147357,-0.035511,0.095478,0.191986,0.184962,-0.071172,-0.095409,0.014755,...,0.041525,0.051206,0.105509,-0.090286,-0.009972,0.026797,-0.035651,0.029559,-0.036017,0.001840
97064,3,-0.139771,0.010162,0.090040,0.118012,-0.053500,0.051304,-0.033344,0.005449,0.128929,...,0.006868,0.068048,-0.036445,-0.009538,-0.025268,-0.015817,-0.033483,-0.013015,0.029489,0.085247
67554,4,-0.153485,0.091630,-0.109631,0.204652,-0.025180,0.021679,0.128964,-0.046514,0.073459,...,-0.010257,-0.046860,0.034887,-0.005999,0.016473,-0.049657,0.040115,0.048062,-0.003286,-0.043701
44898,6,0.074572,0.122940,0.032805,0.140233,0.089041,0.195289,0.278423,0.095618,-0.076779,...,0.083658,-0.012940,-0.023102,0.037525,-0.002086,0.059940,-0.026652,0.095968,-0.037605,0.016972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66046,112867,-0.003698,0.058420,-0.112160,-0.093661,-0.106857,0.237354,0.145180,0.101875,0.155545,...,-0.011074,-0.010610,-0.037254,0.018817,-0.000785,-0.070943,-0.021766,0.015382,0.006920,-0.007916
4505,112868,-0.018848,-0.072739,-0.006864,0.255204,0.036962,0.123991,-0.132854,-0.071834,0.145407,...,0.010167,-0.007480,-0.020405,-0.052083,0.016777,0.072981,0.037118,-0.025302,0.008991,-0.007096
36511,112869,-0.144282,0.135619,0.295813,0.099163,-0.052338,-0.172355,-0.026875,-0.030446,-0.006570,...,-0.010551,0.031673,0.030026,-0.013008,0.081075,-0.013393,-0.001686,-0.056508,-0.004542,0.051326
93558,112870,-0.034649,-0.167996,-0.051505,0.007947,-0.167482,0.003109,0.006069,0.026883,-0.168615,...,-0.005538,0.002468,0.002104,-0.135865,0.040062,-0.077982,-0.021825,-0.057545,-0.097967,0.043246


In [16]:
embs

array([[-0.11119765, -0.1269163 ,  0.11609456, ..., -0.00871609,
        -0.0013299 , -0.03961842],
       [ 0.00525121,  0.14735718, -0.03551126, ...,  0.02955875,
        -0.03601744,  0.00183983],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.14428229,  0.1356188 ,  0.29581288, ..., -0.05650806,
        -0.00454224,  0.05132622],
       [-0.03464856, -0.16799561, -0.05150495, ..., -0.05754471,
        -0.09796727,  0.04324589],
       [-0.01367684,  0.04984036, -0.09001646, ..., -0.02882691,
         0.04850275,  0.03395812]])

In [14]:
exp_feedid = feed_emb['feedid'].unique()
for i in exp_feedid:
    embs[i] = feed_emb[feed_emb['feedid']==i].values[0][1:]

In [128]:
feed_pkl = open(FEATURE_PATH + '/feedid_encoder.pkl', 'wb')
pickle.dump(feedid_lbe, feed_pkl)
feed_pkl.close()

In [17]:
np.save(FEATURE_PATH + '/feed_embedding_100.npy', embs)

# check

In [134]:
feed_info['authorid'].max()

18788

In [2]:
import datatable as dt
import pandas as pd

In [5]:
data = pd.read_csv(USER_ACTION)

In [6]:
val = data[data['date_'] == 14]

In [156]:
train = data[data['date_'] < 14]

In [140]:
val.to_csv(FEATURE_PATH + '/val_data.csv', index=False)

In [157]:
train.to_csv(FEATURE_PATH + '/train_data.csv', index=False)

In [13]:
tag = dt.fread(FEATURE_PATH + '/feed_info_tags_keys_des_seq_len.csv')
tag = tag.to_pandas()

In [22]:
tag[tag['key_seq_len'] >= 11]['key_seq_len'] = 11

/opt/conda/envs/pytorch_py3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [32]:
tag.loc[tag['key_seq_len']>11] = 11

In [33]:
tag[tag['key_seq_len'] > 11]

,feedid,authorid,videoplayseconds,description,ocr,asr,bgm_song_id,bgm_singer_id,manual_keyword_list,machine_keyword_list,...,des_13,des_14,des_15,des_16,des_17,des_18,des_19,tag_seq_len,key_seq_len,des_seq_len


In [34]:
tag.to_csv(FEATURE_PATH + '/feed_info_tags_keys_des_seq_len_11.csv', index=False)

In [5]:
import pandas as pd
feed_embs = pd.read_csv(FEATURE_PATH + '/feed_embeddings_PCA64.csv')


In [6]:
feed_embs

,feedid,feed_embed_0,feed_embed_1,feed_embed_2,feed_embed_3,feed_embed_4,feed_embed_5,feed_embed_6,feed_embed_7,feed_embed_8,...,feed_embed_54,feed_embed_55,feed_embed_56,feed_embed_57,feed_embed_58,feed_embed_59,feed_embed_60,feed_embed_61,feed_embed_62,feed_embed_63
0,46022,0.032595,0.147351,0.027612,0.042160,-0.046727,0.032577,-0.033514,-0.050674,-0.054472,...,0.047478,-0.073130,0.052929,0.029326,-0.053034,0.167495,0.044537,0.069516,-0.027554,-0.020499
1,73903,-0.083516,0.173308,0.066813,-0.153625,-0.063997,0.015803,0.078455,0.062879,-0.135771,...,-0.064933,-0.081195,-0.062658,0.020446,0.004753,0.041112,0.014730,0.026065,-0.067056,0.053601
2,88646,0.033153,0.279646,-0.051001,-0.143725,-0.142205,-0.042946,-0.065661,0.052330,-0.097852,...,-0.005789,-0.018422,0.043514,-0.027066,0.080882,-0.026082,0.033820,0.044866,0.010734,0.052921
3,24381,-0.144089,0.052579,-0.052882,-0.091234,0.162657,-0.037502,-0.073173,0.053545,-0.135384,...,0.004059,-0.001593,-0.006605,-0.098263,-0.012515,0.071267,-0.123089,0.027061,-0.081281,0.090762
4,41542,-0.080312,0.040514,-0.005196,-0.152800,-0.023023,-0.062260,0.047431,0.036471,0.040587,...,0.001817,-0.039569,-0.026686,0.058309,-0.007481,0.013817,-0.102369,0.026615,0.040310,0.048957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106439,14792,-0.089730,-0.079561,0.067023,0.076177,0.094725,0.020977,0.032796,0.055822,-0.052921,...,0.084823,0.044141,0.076324,-0.031074,-0.043780,-0.001242,-0.076630,0.000171,0.069191,0.004851
106440,42659,-0.070165,-0.114127,-0.054354,-0.061213,-0.005316,0.012543,0.049201,-0.125869,0.099338,...,-0.046836,-0.017777,-0.081087,0.043763,-0.067382,0.059749,0.055715,-0.046744,0.066345,-0.008910
106441,66410,-0.035739,-0.135914,0.021624,0.123873,0.021214,0.030240,-0.095187,0.009428,0.089284,...,-0.010393,0.061894,-0.113967,-0.125850,0.042544,0.055391,0.103403,0.063416,0.026527,0.080305
106442,7919,-0.037976,0.050109,-0.039671,0.076309,-0.062639,-0.169440,-0.001236,0.075719,-0.014648,...,-0.011334,-0.030206,0.045613,-0.007666,0.056606,-0.026681,-0.040819,-0.015160,-0.005065,0.047434


In [ ]:
data = dt.fread(SUBMIT_DIR + '/data_128.csv')
data = data.to_pandas()

In [ ]:
train = data[data['date_'] <= 14]
off_train = data[data['date_'] < 14]
val = data[data['date_'] == 14]
test = data[data['date_'] == 15]

In [ ]:
2

In [1]:
!pip install deepctr_torch==0.2.6
!pip install datatable


Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 63 kB 7.0 MB/s eta 0:00:011
     |████████████████████████████████| 454.3 MB 169 kB/s eta 0:00:0101  |██▎                             | 31.6 MB 398 kB/s eta 0:17:42     |██████████████▏                 | 200.5 MB 296 kB/s eta 0:14:17�████████████▍           | 289.8 MB 134.7 MB/s eta 0:00:02     |██████████████████████          | 312.0 MB 170 kB/s eta 0:13:57
     |████████████████████████████████| 462 kB 14.0 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 77.6 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 23.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 87.5 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 80.0 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 13.8 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 41.4 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 81.2 MB/s 

In [2]:
import pickle
hist_file = open(FEATURE_PATH + '/hist_data.pkl', 'rb')
hist_data = pickle.load(hist_file)
hist_file.close()

In [3]:
hist_data['hist_authorid'] = hist_data['hist_authorid'][:, :100]
hist_data['hist_bgm_song_id'] = hist_data['hist_bgm_song_id'][:, :100]
hist_data['hist_bgm_singer_id'] = hist_data['hist_bgm_singer_id'][:, :100]
hist_data['hist_feedid'] = hist_data['hist_feedid'][:, :100]


In [4]:
hist_data['val_seq_len'][np.where(hist_data['val_seq_len'] > 100)] = 100

In [5]:
hist_data['test_seq_len'][np.where(hist_data['test_seq_len'] > 100)] = 100

In [6]:
hist_data['hist_authorid'].shape

(200000, 100)

In [7]:
pkl = open(FEATURE_PATH + '/hist_data_all_begin_100.pkl', 'wb')
pickle.dump(hist_data, pkl)
pkl.close()

In [2]:
import pandas as pd

In [7]:
test = pd.read_csv(FEATURE_PATH + '/online_test.csv')

In [8]:
test

,userid,feedid,date_,device,read_comment,comment,like,click_avatar,forward,follow,favorite,bgm_song_id,bgm_singer_id,authorid,videoplayseconds,real_seq_len
0,140077,50458,15.0,2,0,0,0,0,0,0,0,15157,627,10987,0.000476,58
1,63788,42329,15.0,2,0,0,0,0,0,0,0,0,0,15861,0.000794,34
2,116495,85242,15.0,2,0,0,0,0,0,0,0,0,0,15383,0.000873,3
3,22621,9425,15.0,1,0,0,0,0,0,0,0,1830,15603,17389,0.000595,42
4,35360,11866,15.0,2,0,0,0,0,0,0,0,6411,3907,15695,0.002302,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4252092,122538,51633,15.0,2,0,0,0,0,0,0,0,5942,7445,10138,0.000635,2
4252093,31437,20147,15.0,2,0,0,0,0,0,0,0,0,0,8820,0.000833,8
4252094,2012,89043,15.0,2,0,0,0,0,0,0,0,0,0,18386,0.000992,7
4252095,55470,27238,15.0,2,0,0,0,0,0,0,0,15487,13795,7419,0.002024,7


In [9]:
val = pd.read_csv(FEATURE_PATH + '/offline_val.csv')

In [10]:
val

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite,bgm_song_id,bgm_singer_id,authorid,videoplayseconds,real_seq_len
0,7008,25870,14.0,1,False,False,False,50000.0,61322.0,False,False,False,False,0,0,3076,0.001587,1
1,196225,107066,14.0,2,False,False,False,12833.0,13086.0,False,False,False,False,0,0,10835,0.001786,45
2,22323,29536,14.0,1,False,False,False,16750.0,17766.0,False,False,False,False,15171,6730,14709,0.000516,13
3,196081,10132,14.0,2,False,False,False,0.0,8516.0,False,False,False,False,0,0,2608,0.002302,9
4,195047,107379,14.0,2,False,False,False,20456.0,21562.0,False,False,False,False,24194,9125,4031,0.000635,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6103950,96926,30925,14.0,2,False,False,False,4630.0,6003.0,False,False,False,False,0,0,4601,0.000992,11
6103951,96926,49352,14.0,2,False,False,False,40550.0,42129.0,False,False,False,False,16710,9972,14121,0.000516,11
6103952,96926,14539,14.0,2,False,False,False,6073.0,7960.0,False,False,False,False,0,0,14098,0.002103,11
6103953,96926,42292,14.0,2,False,False,False,0.0,1671.0,False,False,False,False,13986,11862,13716,0.000794,11


In [11]:
test = pd.read_csv(SUBMIT_DIR + '/mmoe_din_1.csv')

In [12]:
test

,userid,feedid,read_comment,like,click_avatar,forward,comment,favorite,follow
0,175282,50458,0.497261,0.506824,0.504572,0.496638,0.493066,0.516813,0.50682
1,80036,42329,0.497261,0.506824,0.504572,0.496638,0.493066,0.516813,0.50682
2,145791,85242,0.497261,0.506824,0.504572,0.496638,0.493066,0.516813,0.50682
3,28430,9425,0.497261,0.506824,0.504572,0.496638,0.493066,0.516813,0.50682
4,44393,11866,0.497261,0.506824,0.504572,0.496638,0.493066,0.516813,0.50682
...,...,...,...,...,...,...,...,...,...
4252092,153322,51633,0.497261,0.506824,0.504572,0.496638,0.493066,0.516813,0.50682
4252093,39430,20147,0.497261,0.506824,0.504572,0.496638,0.493066,0.516813,0.50682
4252094,2524,89043,0.497261,0.506824,0.504572,0.496638,0.493066,0.516813,0.50682
4252095,69629,27238,0.497261,0.506824,0.504572,0.496638,0.493066,0.516813,0.50682


In [14]:
test.isna().sum()

userid          0
feedid          0
read_comment    0
like            0
click_avatar    0
forward         0
comment         0
favorite        0
follow          0
dtype: int64

In [16]:
sub = pd.read_csv(SUBMIT_DIR + '/mmoe_din_all_1.csv')

In [17]:
sub

,userid,feedid,read_comment,like,click_avatar,forward,comment,favorite,follow
0,175282,50458,0.027067,0.030940,0.000843,0.018785,0.001500,0.002798,0.000169
1,80036,42329,0.009208,0.010221,0.013380,0.004300,0.000357,0.002345,0.002311
2,145791,85242,0.002484,0.018620,0.001000,0.001580,0.000083,0.000473,0.000108
3,28430,9425,0.000028,0.004734,0.045315,0.007023,0.000695,0.003132,0.008408
4,44393,11866,0.004206,0.003777,0.001196,0.001323,0.000039,0.000272,0.000111
...,...,...,...,...,...,...,...,...,...
4252092,153322,51633,0.002361,0.012940,0.002433,0.008497,0.000135,0.001388,0.000266
4252093,39430,20147,0.003789,0.011604,0.001303,0.000720,0.000087,0.000135,0.000068
4252094,2524,89043,0.004808,0.016638,0.005725,0.004107,0.000280,0.000822,0.000629
4252095,69629,27238,0.006732,0.011040,0.012780,0.001510,0.000209,0.000311,0.000834


In [19]:
hist_file = open(FEATURE_PATH + '/hist_data_action_all.pkl', 'rb')
hist_data = pickle.load(hist_file)
hist_file.close()

In [23]:
(hist_data['test_seq_len']>100).sum()

9890

In [2]:
train = pd.read_csv(FEATURE_PATH + '/online_train.csv')
train

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite,bgm_song_id,bgm_singer_id,authorid,videoplayseconds,real_seq_len
0,49941,41366,1.0,2,False,False,False,226.0,388.0,False,False,False,False,0,0,3485,0.002064,0
1,182085,29659,1.0,2,False,False,False,37218.0,37545.0,False,False,False,False,6446,17319,17252,0.000595,0
2,182085,96064,1.0,2,False,False,False,64104.0,64499.0,False,False,False,False,3249,15385,11589,0.002262,0
3,182085,26033,1.0,2,False,False,False,42924.0,43234.0,False,False,False,False,0,0,6688,0.000754,0
4,182085,106701,1.0,2,False,False,False,51483.0,52050.0,False,False,False,False,22157,12593,11009,0.001984,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73175506,96926,30925,14.0,2,False,False,False,4630.0,6003.0,False,False,False,False,0,0,4601,0.000992,11
73175507,96926,49352,14.0,2,False,False,False,40550.0,42129.0,False,False,False,False,16710,9972,14121,0.000516,11
73175508,96926,14539,14.0,2,False,False,False,6073.0,7960.0,False,False,False,False,0,0,14098,0.002103,11
73175509,96926,42292,14.0,2,False,False,False,0.0,1671.0,False,False,False,False,13986,11862,13716,0.000794,11


In [4]:
train['real_seq_len_100'] = train['real_seq_len']

In [12]:
train.loc[train.real_seq_len_100 > 100, 'real_seq_len_100'] = 100

In [15]:
test = pd.read_csv(FEATURE_PATH + '/online_test.csv')
test['real_seq_len_100'] = test['real_seq_len']
test.loc[test.real_seq_len_100 > 100, 'real_seq_len_100'] = 100

In [16]:
train.to_csv(FEATURE_PATH + '/online_train_100.csv')
test.to_csv(FEATURE_PATH + '/online_test_100.csv')

In [17]:
off_train = pd.read_csv(FEATURE_PATH + '/offline_train.csv')
off_val = pd.read_csv(FEATURE_PATH + '/offline_val.csv')

off_train['real_seq_len_100'] = off_train['real_seq_len']
off_train.loc[off_train.real_seq_len_100 > 100, 'real_seq_len_100'] = 100

off_val['real_seq_len_100'] = off_val['real_seq_len']
off_val.loc[off_val.real_seq_len_100 > 100, 'real_seq_len_100'] = 100

off_train.to_cFEATURE_PATHFEATURE_PATH + '/offline_train_100.csv')
off_val.to_csv(FEATURE_PATH + '/offline_val_100.csv')

## SVD

In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.sparse.linalg import svds

In [5]:
train = pd.read_csv(FEATURE_PATH + '/online_train_100.csv')

In [7]:
y_list = ['read_comment', 'like', 'click_avatar', 'forward', 'follow', 'comment', 'favorite']
train['action'] = train[y_list].sum(axis=1)

In [10]:
valid = train[train['action'] > 0]

In [ ]:
# concat = train[['feedid', 'song_id']].append(te[['msno', 'song_id']])
user_cnt = 199999
feed_cnt = 106444

In [4]:
pkl = open(FEATURE_PATH + '/feedid_encoder.pkl', 'rb')
feedid_map = pickle.load(pkl)
pkl.close()

In [14]:
train['feedid'] = feedid_map.transform(train['feedid'])

In [16]:
concat = train
member_cnt = 199999
song_cnt = 106444
### 1.构建交互稀疏矩阵
data    = np.ones(len(concat))
msno    = concat['feedid'].values
song_id = concat['userid'].values 
rating = sparse.coo_matrix((data, (msno, song_id)))
rating = (rating > 0) * 1.0

In [26]:
### 2.进行SVD分解 
## svd for user-song pairs
n_component = 100
[u, s, vt] = svds(rating, k=n_component)
print(s[::-1])
s_song = np.diag(s[::-1])

[1683.4837518  1272.67174212  881.23668274  829.26750525  762.40158572
  733.55441181  659.81735496  651.51195342  598.19160779  577.01897209
  552.26048585  546.3042982   505.60864014  485.63015196  477.12472307
  454.01531267  445.27677306  431.39850278  414.78780455  405.38033562
  401.96109618  393.72824906  378.68894266  372.49935053  371.81343795
  366.39215217  362.58642415  357.68369924  343.76918776  337.63237349
  336.92572661  330.04621972  324.68504385  321.38945678  316.72853215
  312.61371933  306.98466716  304.08328657  303.12119709  296.5090528
  294.90536839  287.55006835  286.13498642  283.47251441  279.47085328
  276.83102895  272.69126612  271.06324307  268.54560812  267.41966645]


In [11]:
user = np.load(FEATURE_PATH + '/svd_user_50.npy')
feed = np.load(FEATURE_PATH + '/svd_feed_50.npy')
np.save(FEATURE_PATH + '/svd_user_50.npy', feed)
np.save(FEATURE_PATH + '/svd_feed_50.npy', user)

In [10]:
user.shape

(106444, 50)

In [31]:
np.save(FEATURE_PATH + '/svd_user_50.npy', vt.transpose()[:, ::-1])
np.save(FEATURE_PATH + '/svd_feed_50.npy', u[:, ::-1])

In [ ]:
### 3.生成SVD特征向量
members_topics = pd.DataFrame(u[:, ::-1])
members_topics.columns = ['member_component_%d'%i for i in range(n_component)]
members_topics['msno'] = range(member_cnt)
member = member.merge(members_topics, on='msno', how='right')


In [ ]:
song_topics = pd.DataFrame(vt.transpose()[:, ::-1])
song_topics.columns = ['song_component_%d'%i for i in range(n_component)]
song_topics['song_id'] = range(song_cnt)
song = song.merge(song_topics, on='song_id', how='right')

In [29]:
vt.transpose()[:, ::-1].shape

(199999, 50)

In [ ]:
concat = tr[['msno', 'song_id']].append(te[['msno', 'song_id']])
member_cnt = concat['msno'].max() + 1
song_cnt = concat['song_id'].max() + 1
 

### 1.构建交互稀疏矩阵
data    = np.ones(len(concat))
msno    = concat['msno'].values
song_id = concat['song_id'].values 
rating = sparse.coo_matrix((data, (msno, song_id)))
rating = (rating > 0) * 1.0

### 2.进行SVD分解 
## svd for user-song pairs
n_component = 48 
[u, s, vt] = svds(rating, k=n_component)
print(s[::-1])
s_song = np.diag(s[::-1])

### 3.生成SVD特征向量
members_topics = pd.DataFrame(u[:, ::-1])
members_topics.columns = ['member_component_%d'%i for i in range(n_component)]
members_topics['msno'] = range(member_cnt)
member = member.merge(members_topics, on='msno', how='right')

song_topics = pd.DataFrame(vt.transpose()[:, ::-1])
song_topics.columns = ['song_component_%d'%i for i in range(n_component)]
song_topics['song_id'] = range(song_cnt)
song = song.merge(song_topics, on='song_id', how='right')


In [13]:
data = pd.read_csv(FEATURE_PATH + '/online_train_100.csv')

In [17]:
data['real_seq_len_100'].max()

100

In [16]:
1

1